# Устанавливаем зависимости

In [9]:
# 1. Install PyTorch 1.7.1 with CUDA 11.0 support
%pip install torch torchvision -f https://download.pytorch.org/whl/torch_stable.html

# 2. Install Deep Graph Library (DGL) 0.7.1 for CUDA 11.0
%pip install dgl -f https://data.dgl.ai/wheels/repo.html
%pip install cloudscraper beautifulsoup4 pandas


Looking in links: https://download.pytorch.org/whl/torch_stable.html
Note: you may need to restart the kernel to use updated packages.
Looking in links: https://data.dgl.ai/wheels/repo.html
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Датасет


In [9]:
# 1. Подготовка окружения и авторизация в Kaggle
import os
import json

# Убедимся, что файл kaggle.json лежит в корне проекта
kaggle_json_path = "../kaggle.json"
if not os.path.exists(kaggle_json_path):
    raise FileNotFoundError("Файл 'kaggle.json' не найден в корне проекта.")

with open(kaggle_json_path, "r") as f:
    kaggle_creds = json.load(f)
os.environ["KAGGLE_USERNAME"] = kaggle_creds["username"]
os.environ["KAGGLE_KEY"]      = kaggle_creds["key"]

# Проверяем, существует ли папка
if os.path.exists("./data"):
    # Если папка существует, удаляем все её содержимое
    for root, dirs, files in os.walk("./data", topdown=False):
        for name in files:
            os.remove(os.path.join(root, name))
        for name in dirs:
            os.rmdir(os.path.join(root, name))

# 2. Скачивание датасета из Kaggle
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

dataset_slug = "xblock/ethereum-phishing-transaction-network"
api.dataset_download_files(dataset_slug, path="./data", unzip=False, force=True)

# 3. Распаковка архива (если он в .zip)
import zipfile
archive_path = "./data/ethereum-phishing-transaction-network.zip"
if os.path.exists(archive_path):
    with zipfile.ZipFile(archive_path, "r") as z:
        z.extractall("./data")

Dataset URL: https://www.kaggle.com/datasets/xblock/ethereum-phishing-transaction-network


100%|██████████| 392M/392M [00:00<00:00, 2.33GB/s]

In [1]:
# 4. Загрузка графа из pickle
import pickle
import networkx as nx

pkl_path = "./data/Ethereum Phishing Transaction Network/MulDiGraph.pkl"
with open(pkl_path, "rb") as f:
    G = pickle.load(f)  # networkx.MultiDiGraph

In [3]:
# 5. Вывод основной информации о графе
print("=== Общая информация о графе ===")
print(G)   # число узлов, число ребер, тип графа

def graph_summary(g: "nx.Graph") -> str:
    summary = [
        f"{type(g).__name__}",
        f"directed={g.is_directed()}",
        f"nodes={g.number_of_nodes()}",
        f"edges={g.number_of_edges()}",
    ]
    # пример дополнительной метрики
    avg_deg = sum(dict(g.degree()).values()) / g.number_of_nodes()
    summary.append(f"avg_degree={avg_deg:.2f}")
    return " | ".join(summary)

print(graph_summary(G))


# 6. Атрибуты узлов и ребер
# Пример: первые 5 узлов и их атрибуты
print("\n=== Пример узлов с атрибутами ===")
for node, attrs in list(G.nodes(data=True))[:5]:
    print(f"{node}: {attrs}")

# Пример: первые 5 ребер и их атрибуты
print("\n=== Пример ребер с атрибутами ===")
for u, v, attrs in list(G.edges(data=True))[:5]:
    print(f"{u} -> {v}: {attrs}")

# 7. Подсчёт помеченных мошеннических (phishing) узлов
fraud_nodes = [n for n, d in G.nodes(data=True) if d.get("isp", 0) == 1]
print(f"\nЧисло узлов с isp=1 (phishing): {len(fraud_nodes)} / {G.number_of_nodes()}")

# 8. Вывод распределения степени (degree) для первой оценки
degrees = dict(G.degree())
deg_values = list(degrees.values())
print(f"\nМинимальная степень: {min(deg_values)}")
print(f"Максимальная степень: {max(deg_values)}")
print(f"Средняя степень: {sum(deg_values)/len(deg_values):.2f}")

=== Общая информация о графе ===
MultiDiGraph with 2973489 nodes and 13551303 edges
MultiDiGraph | directed=True | nodes=2973489 | edges=13551303 | avg_degree=9.11

=== Пример узлов с атрибутами ===
0x1f1e784a61a8ca0a90250bcd2170696655b28a21: {'isp': 0}
0x1266f8b9e4dffc9e2f719bf51713f7e714516861: {'isp': 0}
0xbbfaf27674c2eb5d13edc58a40081248d13dcfeb: {'isp': 1}
0x256fc19e9d8f5be0d451841f218289d1adbbaaa3: {'isp': 0}
0xb50d0c4cb2c29cc232c96a59e9c65eb82914ec75: {'isp': 0}

=== Пример ребер с атрибутами ===
0x1f1e784a61a8ca0a90250bcd2170696655b28a21 -> 0x1266f8b9e4dffc9e2f719bf51713f7e714516861: {'amount': 2.3446233, 'timestamp': 1526454086.0}
0x1f1e784a61a8ca0a90250bcd2170696655b28a21 -> 0x806ceb189d36700a97f4e7ecd4fb6c95f2c5a3de: {'amount': 0.07, 'timestamp': 1504461965.0}
0x1f1e784a61a8ca0a90250bcd2170696655b28a21 -> 0x806ceb189d36700a97f4e7ecd4fb6c95f2c5a3de: {'amount': 0.052111, 'timestamp': 1504473420.0}
0x1f1e784a61a8ca0a90250bcd2170696655b28a21 -> 0x3ec4688db6bf8464b0bef30ec2ca7afc

In [5]:
import random
import networkx as nx

# Для воспроизводимости
random.seed(42)

# 1. Получаем списки мошеннических и немошеннических узлов
fraud_nodes = [n for n, d in G.nodes(data=True) if d.get("isp", 0) == 1]
normal_nodes = [n for n in G.nodes() if n not in fraud_nodes]

# 2. Случайным образом выбираем столько же нормальных узлов, сколько мошеннических
sampled_normal = random.sample(normal_nodes, len(fraud_nodes))

# 3. Объединяем все выбранные узлы
selected = set(fraud_nodes + sampled_normal)

# 4. Собираем всех их соседей (входящие + исходящие)
neighbors = set()
for node in selected:
    neighbors.update(G.predecessors(node))
    neighbors.update(G.successors(node))

# 5. Итоговый набор узлов субграфа
sub_nodes = selected.union(neighbors)

# 6. Строим индуцированный субграф и делаем копию
G_sub = G.subgraph(sub_nodes).copy()

# 7. Выводим информацию о получившемся субграфе
print("Субграф:") 
print(G_sub)  # ожидается около 9629 узлов и 386612 ребер

def graph_summary(g: "nx.Graph") -> str:
    summary = [
        f"{type(g).__name__}",
        f"directed={g.is_directed()}",
        f"nodes={g.number_of_nodes()}",
        f"edges={g.number_of_edges()}",
    ]
    # пример дополнительной метрики
    avg_deg = sum(dict(g.degree()).values()) / g.number_of_nodes()
    summary.append(f"avg_degree={avg_deg:.2f}")
    return " | ".join(summary)

print(graph_summary(G_sub))

Субграф:
MultiDiGraph with 30857 nodes and 1382854 edges
MultiDiGraph | directed=True | nodes=30857 | edges=1382854 | avg_degree=89.63


# Etherscan labels

In [11]:
# Если еще не стоит: pip install cloudscraper beautifulsoup4 pandas

import cloudscraper
from bs4 import BeautifulSoup
import pandas as pd
import time
import networkx as nx
from collections import Counter

# 1) Создаем сессию-скрапер, которая обходит Cloudflare
scraper = cloudscraper.create_scraper(
    browser={
        'browser': 'firefox', 
        'platform': 'windows', 
        'mobile': False
    }
)

# 2) Список нужных нам меток
labels = [
    "exchange",
    "token-contract",
    "gaming-tokens",
    "gambling-accounts",
    "ico-wallets",
    "wallet-app",
    "cold-wallet",
]

records = []
for label in labels:
    print(f"→ собираем «{label}» …")
    page = 1
    while True:
        url = f"https://etherscan.io/accounts/label/{label}?p={page}"
        resp = scraper.get(url)
        if resp.status_code != 200:
            print(f"  стоп, код {resp.status_code}")
            break
        soup = BeautifulSoup(resp.text, "html.parser")
        table = soup.select_one("table.table")
        if not table or not table.tbody or not table.tbody.find_all("tr"):
            break
        for tr in table.tbody.find_all("tr"):
            addr = tr.find_all("td")[1].get_text(strip=True)
            records.append((addr, label))
        page += 1
        time.sleep(1)  # polite crawl delay

# 3) Собираем dataframe и сохраняем
df_node_types = pd.DataFrame(records, columns=["address","node_type"]).drop_duplicates()
df_node_types.to_csv("./data/node_types.csv", index=False)
print("\nГотово, всего записей:", len(df_node_types))

# 4) Аннотируем субграф G_sub из предыдущего шага
type_map = dict(zip(df_node_types["address"], df_node_types["node_type"]))
default_type = "account"
nx.set_node_attributes(
    G_sub,
    {n: type_map.get(n, default_type) for n in G_sub.nodes()},
    name="node_type"
)

# 5) Строим и выводим таблицу по Table 3
type_counts = Counter(nx.get_node_attributes(G_sub, "node_type").values())
total = G_sub.number_of_nodes()
df_types = (
    pd.DataFrame([
        {
            "Node Type": t,
            "Number of nodes": c,
            "Percentage": c/total*100
        } for t, c in type_counts.items()
    ])
    .sort_values("Number of nodes", ascending=False)
    .reset_index(drop=True)
)
print(df_types.to_string(index=False))

→ собираем «exchange» …
  стоп, код 403
→ собираем «token-contract» …
  стоп, код 403
→ собираем «gaming-tokens» …
  стоп, код 403
→ собираем «gambling-accounts» …
  стоп, код 403
→ собираем «ico-wallets» …
  стоп, код 403
→ собираем «wallet-app» …
  стоп, код 403
→ собираем «cold-wallet» …
  стоп, код 403

Готово, всего записей: 0
Node Type  Number of nodes  Percentage
  account            30857       100.0
